In [20]:
import json
import os
import shutil
import sqlite3
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret

from logzero import logger
from requests import get
import calendar
import regex as re
import pandas as pd

In [21]:
with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [22]:
query_zip = """
SELECT *

FROM program

LEFT JOIN zipcode on program.state_id == zipcode.state_id

WHERE is_entire_state == 1
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

state_df = pd.read_sql_query(query_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [23]:
state_df.shape

(1677080, 30)

In [24]:
query_city_zip = """
SELECT *

FROM program_city

LEFT JOIN program

on program_city.program_id == program.id

LEFT JOIN zipcode on program_city.city_id == zipcode.city_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

city_df = pd.read_sql_query(query_city_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [25]:
city_df.shape

(14048, 32)

In [26]:
query_county_zip = """
SELECT *

FROM program_county

LEFT JOIN program

on program_county.program_id == program.id

LEFT JOIN zipcode on program_county.county_id == zipcode.county_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

county_df = pd.read_sql_query(query_county_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [27]:
county_df.shape

(4889, 32)

In [28]:
query_utility_zip = """
SELECT *

FROM program_utility

LEFT JOIN program

on program_utility.program_id == program.id

LEFT JOIN utility_zipcode

on program_utility.utility_id == utility_zipcode.utility_id

LEFT JOIN zipcode on utility_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

utility_df = pd.read_sql_query(query_utility_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [29]:
utility_df.shape

(434875, 34)

In [30]:
query_zip_zip = """
SELECT *

FROM program_zipcode

LEFT JOIN program

on program_zipcode.program_id == program.id

INNER JOIN zipcode on program_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

zip_df = pd.read_sql_query(query_zip_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [31]:
zip_df.shape

(36, 32)

In [32]:
county_df = county_df.loc[:,~county_df.columns.duplicated()]
city_df = city_df.loc[:,~city_df.columns.duplicated()]
state_df = state_df.loc[:,~state_df.columns.duplicated()]
zip_df = zip_df.loc[:,~zip_df.columns.duplicated()]
utility_df = utility_df.loc[:,~utility_df.columns.duplicated()]

keep_cols = list(
    set(zip_df.columns).intersection(
    set(utility_df.columns)).intersection(
    set(county_df.columns)).intersection(
    set(city_df.columns)).intersection(
    set(state_df.columns))
)



dsire_by_zip = pd.concat([zip_df[keep_cols],
                          utility_df[keep_cols],
                          county_df[keep_cols],
                          city_df[keep_cols],
                          state_df[keep_cols]])

In [33]:
dsire_by_zip.columns

Index(['updated_ts', 'created_by_user_id', 'start_date_text', 'state_id',
       'latitude', 'county_id', 'published', 'city_id', 'fromSir',
       'implementing_sector_id', 'summary', 'websiteurl', 'budget',
       'program_category_id', 'is_entire_state', 'end_date', 'zipcode',
       'end_date_text', 'start_date', 'code', 'name', 'longitude',
       'program_type_id', 'created_ts', 'additional_technologies', 'id',
       'fundingsource', 'administrator'],
      dtype='object')

In [34]:
query_tech = """
SELECT * 

FROM program_technology

WHERE program_technology.technology_id == 7
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

tech_df = pd.read_sql_query(query_tech, conn_proj)

conn_proj.close

<function Connection.close>

In [35]:
check_list = tech_df.program_id.to_list()
dsire_by_zip.head()

,updated_ts,created_by_user_id,start_date_text,state_id,latitude,county_id,published,city_id,fromSir,implementing_sector_id,...,start_date,code,name,longitude,program_type_id,created_ts,additional_technologies,id,fundingsource,administrator
0,2020-05-28 20:50:44,492,None,27,44.0,1680.0,1,17335.0,0,3,...,None,MN57F,Rochester Public Utilities - Commercial and In...,-92.0,88,2006-08-09 00:00:00,None,1946,None,Rochester Public Utilities
1,2019-05-13 13:45:11,534,None,54,39.0,247.0,1,6019.0,0,3,...,None,VA32F,Columbia Gas of Virginia - Home Savings Rebate...,-78.0,88,2010-05-06 00:00:00,None,4127,None,None
2,2019-05-13 13:45:11,534,None,54,39.0,249.0,1,6022.0,0,3,...,None,VA32F,Columbia Gas of Virginia - Home Savings Rebate...,-77.0,88,2010-05-06 00:00:00,None,4127,None,None
3,2019-05-13 13:45:11,534,None,54,39.0,250.0,1,6022.0,0,3,...,None,VA32F,Columbia Gas of Virginia - Home Savings Rebate...,-77.0,88,2010-05-06 00:00:00,None,4127,None,None
4,2019-05-13 13:45:11,534,None,54,39.0,249.0,1,6022.0,0,3,...,None,VA32F,Columbia Gas of Virginia - Home Savings Rebate...,-77.0,88,2010-05-06 00:00:00,None,4127,None,None


In [36]:
dsire_PV = dsire_by_zip[dsire_by_zip.id.isin(check_list)]
dsire_PV.start_date.unique()

array([None, '2015-01-26 20:00:00', '2010-01-01 05:00:00',
       '1997-01-26 21:59:32', '2012-10-01 04:00:00',
       '1997-12-16 05:00:00', '2015-01-26 05:00:00',
       '2007-01-01 00:00:00', '2010-08-12 04:00:00',
       '2017-01-01 05:00:00', '2003-05-01 04:00:00',
       '1999-01-26 05:00:00', '2004-10-01 00:00:00',
       '2004-06-01 04:00:00', '2004-01-01 05:00:00',
       '2015-01-26 20:05:00', '2015-09-05 00:00:00',
       '2005-10-01 00:00:00', '2006-03-01 05:00:00',
       '2005-12-01 00:00:00', '2006-01-01 05:00:00',
       '2021-02-01 05:00:00', '2006-06-01 04:00:00',
       '2006-07-01 04:00:00', '2007-01-01 05:00:00',
       '2010-10-01 00:00:00', '2006-10-01 00:00:00',
       '2008-01-25 05:00:00', '2007-09-01 04:00:00',
       '2007-10-01 04:00:00', '2008-02-14 05:00:00',
       '2013-05-29 04:00:00', '2008-01-09 00:00:00',
       '2008-07-01 04:00:00', '2008-08-08 04:00:00',
       '2011-01-01 00:00:00', '2008-12-15 00:00:00',
       '2009-10-01 04:00:00', '2009-03-0

In [37]:
dsire_PV['start_date'] = dsire_PV.start_date.fillna(dsire_PV.created_ts)
dsire_PV['end_date'] = dsire_PV.end_date.fillna(datetime.strptime(cfg["max_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
diff_df = dsire_PV.dropna(subset = ['start_date', 'zipcode']).reset_index()
diff_df['zipcode'] = diff_df.zipcode.apply(lambda n: str(int(n)).zfill(5))
diff_df.head()

<ipython-input-37-dca77c2a7bf5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsire_PV['start_date'] = dsire_PV.start_date.fillna(dsire_PV.created_ts)
<ipython-input-37-dca77c2a7bf5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsire_PV['end_date'] = dsire_PV.end_date.fillna(datetime.strptime(cfg["max_date"], "%d-%b-%Y (%H:%M:%S.%f)"))


,index,updated_ts,created_by_user_id,start_date_text,state_id,latitude,county_id,published,city_id,fromSir,...,start_date,code,name,longitude,program_type_id,created_ts,additional_technologies,id,fundingsource,administrator
0,0,2004-11-22 00:00:00,534,None,7,40.0,2814.0,0,25521.0,0,...,2003-09-05 00:00:00,CO20R,Xcel Energy - Net Metering,-105.0,37,2003-09-05 00:00:00,None,9,None,None
1,1,2004-11-22 00:00:00,534,None,7,40.0,2814.0,0,25521.0,0,...,2003-09-05 00:00:00,CO20R,Xcel Energy - Net Metering,-105.0,37,2003-09-05 00:00:00,None,9,None,None
2,2,2004-11-22 00:00:00,534,None,7,40.0,2814.0,0,25521.0,0,...,2003-09-05 00:00:00,CO20R,Xcel Energy - Net Metering,-105.0,37,2003-09-05 00:00:00,None,9,None,None
3,3,2004-11-22 00:00:00,534,None,7,40.0,2814.0,0,25521.0,0,...,2003-09-05 00:00:00,CO20R,Xcel Energy - Net Metering,-105.0,37,2003-09-05 00:00:00,None,9,None,None
4,4,2004-11-22 00:00:00,534,None,7,40.0,2814.0,0,25521.0,0,...,2003-09-05 00:00:00,CO20R,Xcel Energy - Net Metering,-105.0,37,2003-09-05 00:00:00,None,9,None,None


In [39]:
# import json
# with open("../plotly_zips.geojson") as f:
#     zips = json.load(f)

In [ ]:
# import pandas as pd
# import plotly
# import plotly.graph_objs as go
# import plotly.offline as offline
# from plotly.graph_objs import *
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# init_notebook_mode(connected=True)

### Create chloropleth of policy types over time

Need to have Month and year ranging from first date to last for every zip code
This will need to tally the number of programs within the zip code existing during that month
Use start date - add 1 to that month and each month after (for policy count)
Use end date - subtract 1 from that month and each month after (for policy count)

If tracking other metrics, use different logic to aggregate, but accumulate per same logic pertaining to month

In [ ]:
# all_zips = [i['id'] for i in zips['features']]

In [ ]:
# with open('../data/uszips.txt', 'w') as filehandle:
#     filehandle.writelines("%s\n" % place for place in all_zips)

In [40]:
diff_df.columns

Index(['index', 'updated_ts', 'created_by_user_id', 'start_date_text',
       'state_id', 'latitude', 'county_id', 'published', 'city_id', 'fromSir',
       'implementing_sector_id', 'summary', 'websiteurl', 'budget',
       'program_category_id', 'is_entire_state', 'end_date', 'zipcode',
       'end_date_text', 'start_date', 'code', 'name', 'longitude',
       'program_type_id', 'created_ts', 'additional_technologies', 'id',
       'fundingsource', 'administrator'],
      dtype='object')

In [41]:
program_map_df = diff_df[['program_type_id','zipcode','start_date','end_date']]
program_map_df['s_d'] = pd.to_datetime(program_map_df.start_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
program_map_df['e_d'] = pd.to_datetime(program_map_df.end_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))

program_map_df.head()

<ipython-input-41-f75d12fb4149>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  program_map_df['s_d'] = pd.to_datetime(program_map_df.start_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
<ipython-input-41-f75d12fb4149>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  program_map_df['e_d'] = pd.to_datetime(program_map_df.end_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))


,program_type_id,zipcode,start_date,end_date,s_d,e_d
0,37,80002,2003-09-05 00:00:00,2019-12-31 00:00:00,2044 days,8005 days
1,37,80003,2003-09-05 00:00:00,2019-12-31 00:00:00,2044 days,8005 days
2,37,80004,2003-09-05 00:00:00,2019-12-31 00:00:00,2044 days,8005 days
3,37,80005,2003-09-05 00:00:00,2019-12-31 00:00:00,2044 days,8005 days
4,37,80007,2003-09-05 00:00:00,2019-12-31 00:00:00,2044 days,8005 days


In [43]:
program_map_df["s_d"] = program_map_df.s_d.dt.days
program_map_df["e_d"] = program_map_df.e_d.dt.days
program_map_df.head()

<ipython-input-43-39fae3f67d15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  program_map_df["s_d"] = program_map_df.s_d.dt.days
<ipython-input-43-39fae3f67d15>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  program_map_df["e_d"] = program_map_df.e_d.dt.days


,program_type_id,zipcode,start_date,end_date,s_d,e_d
0,37,80002,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005
1,37,80003,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005
2,37,80004,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005
3,37,80005,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005
4,37,80007,2003-09-05 00:00:00,2019-12-31 00:00:00,2044,8005


In [49]:
program_map_df.dtypes

program_type_id     int64
zipcode            object
start_date         object
end_date           object
s_d                 int64
e_d                 int64
dtype: object

In [54]:
def keys_exists(element, *keys):
    '''
    Check if *keys (nested) exists in `element` (dict).
    '''
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True



from collections import defaultdict
program_map = defaultdict(dict)

# for p_id in program_map_df.program_type_id.unique():
#     """df = program_map_df[program_map_df.program_type_id == p_id]
#     z_d = {}
#     for z in df.zipcode.unique():
#         df_2 = df[df.zipcode == z]
#         z_d[z] = tuple(zip())
#     ls = list(zip(df.s_d,df.e_d))"""
    
for a, b, c, d in zip(program_map_df.program_type_id,program_map_df.zipcode,program_map_df.s_d,program_map_df.e_d):
    
    if keys_exists(program_map, a, b):
        program_map[a][b].append([c,d])
    else:
        program_map[a][b] = [[c,d]]
    
    
program_map[37]['80002']

[[2044, 8005], [2512, 8005]]

In [55]:
import pickle

with open('../data/program_zip_dates.p', 'wb') as fp:
    pickle.dump(program_map, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data_zips = []

# open file and read the content in a list
with open('../data/datazips.txt', 'r') as filehandle:
    filecontents = filehandle.readlines()

    for line in filecontents:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        data_zips.append(current_place)
        
data_zips = sorted(data_zips)

In [ ]:
diff_df['install_month'] = pd.to_datetime(diff_df.start_date).dt.strftime('%Y-%m')
from pandas.tseries.offsets import MonthEnd
diff_df['install_month'] = pd.to_datetime(diff_df['install_month']) + MonthEnd(1)


diff_df['end_month'] = pd.to_datetime(diff_df.end_date).dt.strftime('%Y-%m')
diff_df['end_month'] = pd.to_datetime(diff_df['end_month']) + MonthEnd(1)

diff_df.head()

In [ ]:
program_types = diff_df.program_type_id.unique()

In [ ]:
program_dict = {}
for p_id in program_types:
    df = diff_df[diff_df.program_type_id == p_id]
    ls = df.sta
    
    program_dict[p_id] = dict(zip(df.zipcode,df.start_date))
    
import pickle

with open('../data/program_dates.p', 'wb') as fp:
    pickle.dump(program_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

# policy_dict = {}
# for zc in diff_df.zipcode.unique():
#     df = diff_df[diff_df.zipcode == zc]
#     for p_id in df.program_type_id.unique():
#         sub = df[df.program_type_id == p_id]
#         policy_dict[zc] = {p_id: (list(sub.start_date)[0], list(sub.end_date)[0])}
        


In [ ]:
#min_yr = diff_df.install_month.min().year
#min_mo = diff_df.install_month.min().month
min_yr = pd.to_datetime('1998-04-30 00:00:00').year
min_mo = pd.to_datetime('1998-04-30 00:00:00').month

diff_df['num_months'] = diff_df.install_month.apply(
    lambda d: (d.year - min_yr) * 12 + (d.month - min_mo)
)

diff_df['num_end_months'] = diff_df.end_month.apply(
    lambda d: (d.year - min_yr) * 12 + (d.month - min_mo)
)

diff_df_mo = diff_df[diff_df.num_months > 0]

In [ ]:
#attempt to re-run above as dict comprehension to save time
diff_df_mo['s_e'] = diff_df_mo.apply(lambda x: (x.num_months, x.num_end_months), axis=1)
date_df = diff_df_mo.groupby(['zipcode','program_type_id']).s_e.apply(list).reset_index()

date_df.head()

In [ ]:
policy_dict = {}
for zc in date_df.zipcode.unique():
    df = date_df[date_df.zipcode == zc]
    policy_dict[zc] = dict(zip(df.program_type_id, df.s_e))

In [ ]:
policy_dict['91708']

In [ ]:
diff_df_mo[(diff_df_mo.zipcode == '78241') & (diff_df_mo.program_type_id == 37)]

In [ ]:
nm_df = date_df[date_df.program_type_id == 37]

nm_df['earliest_instance'] = nm_df.s_e.apply(lambda s: min([x[0] for x in s]))

nm_df.sort_values(by = 'earliest_instance', ascending = False)

In [ ]:
nm_df['earliest_instance'] = nm_df['earliest_instance'].astype(float)

In [ ]:
nm_df['zipcode'] = nm_df['zipcode'].astype('string')
nm_df.head()

In [ ]:
earliest_dict = {}
for p_id in program_types:
    p_df = date_df[date_df.program_type_id == p_id]
    p_df['earliest_instance'] = p_df.s_e.apply(lambda s: min([x[0] for x in s]))
    p_df['earliest_instance'] = p_df['earliest_instance'].astype(float)
    p_df['zipcode'] = p_df['zipcode'].astype('string')
    
    earliest_dict[p_id] = dict(zip(p_df.zipcode,p_df.earliest_instance))
    
import pickle

with open('../data/earliest_dates.p', 'wb') as fp:
    pickle.dump(earliest_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import json
import plotly.express as px


with open("../plotly_zips.geojson") as f:
    zips = json.load(f)

fig = px.choropleth_mapbox(nm_df, geojson=zips, locations='zipcode', color='earliest_instance',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5
                          )


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
zips

In [ ]:
diff_df_mo.to_pickle("../data/diff_df_mo.pkl")

In [ ]:
#split out db creation into new notebook


#create dependencies for database
#filter out territories and zips w/out install data


#output new db with similarity connections for zip codes
#for a given policy and zip code, include dates from prior policy implementation until next implementation
## may need to filter out certain policy types

#is it worthwhile to create a regression including policy columns?

#establish zip codes of interest in cfg file
#94025 - look at policy implementations prior to 2009

#identify most similar by state

import os
os.chdir("../../solar-697/ru")
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

In [ ]:
diff_df.columns